# Generate Requirements file
This document aims to generate a requirements.txt file that we can use later on. It can be rerun so we can constantly keep this up to date.

We install requirements.txt using ```pip install -r requirements.txt```

In [1]:
import os
import nbformat

In [2]:
def scan_files(directory, extension):
    # Look for files
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(extension):
                yield os.path.join(root, file)

In [3]:
def parse_requirements_file(filename):
    # parse files
    with open(filename) as f:
        return set(line.strip() for line in f)

In [4]:
def parse_ipynb_file(filename):
    # handle notebooks
    with open(filename) as f:
        nb = nbformat.read(f, nbformat.NO_CONVERT)

    requirements = set()
    for cell in nb.cells:
        if cell.cell_type == "code":
            for line in cell.source.splitlines():
                if line.startswith("!pip install"):
                    # Extract the package name from the line and add it to the set of requirements
                    package = line.split("!pip install ")[1]
                    requirements.add(package)

    return requirements

In [5]:
def main():
    # generate the file
    requirements = set()

    # Scan for all .py and .ipynb files in the current directory and subdirectories
    for file in scan_files(".", (".py", ".ipynb")):
        if file.endswith(".ipynb"):
            # If the file is a Jupyter notebook, parse it for !pip installs
            requirements |= parse_ipynb_file(file)
        else:
            with open(file) as f:
                contents = f.read()

            # Look for pip installs in the file's contents
            for line in contents.splitlines():
                if line.startswith("!pip install"):
                    # Extract the package name from the line and add it to the set of requirements
                    package = line.split("!pip install ")[1]
                    requirements.add(package)

    # Parse any existing requirements.txt file to add to the set of requirements
    if os.path.exists("requirements.txt"):
        requirements |= parse_requirements_file("requirements.txt")

    # Write the final set of requirements to a new requirements.txt file
    with open("requirements.txt", "w") as f:
        f.write("\n".join(sorted(requirements)))

In [6]:
if __name__ == "__main__":
    main()